In [9]:
import h5py
import eeg_processing as lib
import h5py_mat_example as load_edf


ModuleNotFoundError: No module named 'h5py'

In [4]:
# Read the EDF_RawData.mat file and print its contents

load_edf.load_eeg_data_from_h5_mat('EDF_RawData.mat')

NameError: name 'load_edf' is not defined